In [ ]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"

# Create driver
Also set the DB_NAME

In [ ]:
import os
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience # Python GDS client

In [ ]:
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))

In [ ]:
# Get a list of all databases (except system and fabric)
gds.set_database("system")
databases = gds.run_cypher("show databases yield name where name <> 'fabric' and name <> 'system' return name")
databases

In [ ]:
# Drop my failed indexes and constraints
# for database in databases["name"]:
#     print(database)
#     gds.set_database(database)
#     indexes = gds.run_cypher("""
#         show indexes yield name, type, labelsOrTypes, properties, uniqueness, createStatement
#         where type = "RANGE" and (uniqueness = "UNIQUE" or uniqueness = "NONUNIQUE")
#         return name, type, labelsOrTypes, properties, uniqueness, createStatement
#     """)
#     for index in indexes[ indexes['uniqueness'] == 'UNIQUE' ] [['name','uniqueness']].to_dict('records'):
#         gds.run_cypher("drop constraint {name}".format(name = index["name"]))
#     for index in indexes[ indexes['uniqueness'] == 'NONUNIQUE' ] [['name','uniqueness']].to_dict('records'):
#          gds.run_cypher("drop index {name}".format(name = index["name"]))

In [ ]:
# Create indexes and constraints needed for migrating to neo4j 5
for database in databases["name"]:
    print(database)
    gds.set_database(database)
    indexes = gds.run_cypher("""
        show indexes yield name, type, labelsOrTypes, properties, uniqueness, createStatement
        where type = "BTREE" and (uniqueness = "UNIQUE" or uniqueness = "NONUNIQUE")
        return name, type, labelsOrTypes, properties, uniqueness, createStatement
    """)
    for index in indexes[ indexes['uniqueness'] == 'UNIQUE' ] [['name','uniqueness', 'labelsOrTypes', 'createStatement', 'properties']].to_dict('records'):
        if 'NODE KEY' in index['createStatement']:
            gds.run_cypher("CREATE CONSTRAINT IF NOT EXISTS FOR (n:{label}) REQUIRE ({properties}) IS NODE KEY OPTIONS {{indexProvider: 'range-1.0'}}".format(
                label = index["labelsOrTypes"][0],
                properties = ','.join([ 'n.' + i for i in index['properties'] ])
            )) 
        else:
            gds.run_cypher("CREATE CONSTRAINT IF NOT EXISTS FOR (n:{label}) REQUIRE ({properties}) IS UNIQUE OPTIONS {{indexProvider: 'range-1.0'}}".format(
                label = index["labelsOrTypes"][0],
                properties = ','.join([ 'n.' + i for i in index['properties'] ])
            )) 
    for index in indexes[ indexes['uniqueness'] != 'UNIQUE' ] [['name','uniqueness', 'labelsOrTypes', 'createStatement', 'properties']].to_dict('records'):
        gds.run_cypher("CREATE INDEX IF NOT EXISTS FOR (n:{label}) ON ({properties})".format(
                label = index["labelsOrTypes"][0],
                properties = ','.join([ 'n.' + i for i in index['properties'] ])
            )) 